# Assignment 1: Programming for Analytics (Python)

**Notebook:** `A1_SecB_B2025100.ipynb`

## PLEASE READ

- All solutions are implemented as reusable functions or classes with docstrings.
- Demo cells run deterministic scenarios so the notebook can execute from top to bottom without manual input.
- To explore interactively, call the helper functions without scripted input—each function accepts optional callbacks or iterables for user interaction.

In [ ]:
from __future__ import annotations

import itertools
import math
import random
import re
import statistics
import textwrap
from collections import Counter
from dataclasses import dataclass, field
from typing import Callable, Dict, Iterable, List, Optional, Sequence, Tuple


## 1. Word Counter

The implementation below analyses the supplied text and reports:

- total words, sentences, and characters
- lexical density, average word length, and an estimated Flesch-Kincaid reading grade
- the most common words (case-insensitive) with their frequencies

All calculations are wrapped in a `TextAnalysis` dataclass that offers a friendly `describe()` helper for formatted output.

In [ ]:
@dataclass
class TextAnalysis:
    """Container for metrics returned by :func:`analyze_text`."""

    words: List[str]
    sentences: List[str]
    characters_with_spaces: int
    characters_without_spaces: int
    average_word_length: float
    lexical_density: float
    most_common_words: List[Tuple[str, int]]
    readability_grade: float

    def as_dict(self) -> Dict[str, object]:
        return {
            "word_count": len(self.words),
            "sentence_count": len(self.sentences),
            "characters_with_spaces": self.characters_with_spaces,
            "characters_without_spaces": self.characters_without_spaces,
            "average_word_length": round(self.average_word_length, 2),
            "lexical_density": round(self.lexical_density, 3),
            "readability_grade": round(self.readability_grade, 2),
            "most_common_words": self.most_common_words,
        }

    def describe(self) -> str:
        lines = ["Text Analysis Summary:"]
        summary = self.as_dict()
        for key, value in summary.items():
            lines.append(f"- {key.replace('_', ' ').title()}: {value}")
        lines.append("- Example sentences:")
        for idx, sentence in enumerate(self.sentences[:3], start=1):
            lines.append(f"  {idx}. {sentence}")
        if len(self.sentences) > 3:
            lines.append("  …")
        return "\n".join(lines)


# Helper for readability computation
def _count_syllables(word: str) -> int:
    """Naive syllable counter used for Flesch-Kincaid grade estimation."""

    word = word.lower()
    vowels = "aeiouy"
    syllables = 0
    previous_was_vowel = False
    for char in word:
        if char in vowels:
            if not previous_was_vowel:
                syllables += 1
            previous_was_vowel = True
        else:
            previous_was_vowel = False
    if word.endswith("e") and syllables > 1:
        syllables -= 1
    return max(syllables, 1)


def analyze_text(text: str, *, top_n: int = 5) -> TextAnalysis:
    """Analyse the provided text and return detailed metrics.

    Parameters
    ----------
    text:
        Input paragraph or document.
    top_n:
        Number of most common words to keep in the summary.
    """

    sentences = [s.strip() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]
    words = re.findall(r"[A-Za-z']+", text.lower())
    characters_with_spaces = len(text)
    characters_without_spaces = len(re.sub(r"\s+", "", text))
    avg_word_length = statistics.mean(len(word) for word in words) if words else 0.0
    lexical_density = len(set(words)) / len(words) if words else 0.0
    common_words = Counter(words).most_common(top_n)

    if sentences and words:
        syllables = sum(_count_syllables(word) for word in words)
        words_per_sentence = len(words) / len(sentences)
        syllables_per_word = syllables / len(words)
        readability_grade = 0.39 * words_per_sentence + 11.8 * syllables_per_word - 15.59
    else:
        readability_grade = 0.0

    return TextAnalysis(
        words=words,
        sentences=sentences,
        characters_with_spaces=characters_with_spaces,
        characters_without_spaces=characters_without_spaces,
        average_word_length=avg_word_length,
        lexical_density=lexical_density,
        most_common_words=common_words,
        readability_grade=readability_grade,
    )


In [ ]:
sample_text = (
    "Python empowers analysts to move from spreadsheets to reproducible, auditable work."
    " In this assignment we explore classic programming exercises and extend them with analytics-inspired insights."
)
analysis = analyze_text(sample_text)
print(analysis.describe())


Text Analysis Summary:
- Word Count: 26
- Sentence Count: 2
- Characters With Spaces: 193
- Characters Without Spaces: 169
- Average Word Length: 6.35
- Lexical Density: 0.962
- Readability Grade: 13.53
- Most Common Words: [('to', 2), ('python', 1), ('empowers', 1), ('analysts', 1), ('move', 1)]
- Example sentences:
  1. Python empowers analysts to move from spreadsheets to reproducible, auditable work.
  2. In this assignment we explore classic programming exercises and extend them with analytics-inspired insights.


## 2. Number Guessing Game

The guessing game now supports three difficulty presets, range narrowing hints, and a warm/cold indicator based on how close the player is.

For automated testing, we have scripted guesses; in interactive sessions, we can supply your own `input_func` that reads from the keyboard.

In [ ]:
@dataclass
class AttemptFeedback:
    guess: int
    feedback: str
    remaining_low: int
    remaining_high: int


@dataclass
class GuessingGameResult:
    target: int
    success: bool
    attempts_used: int
    history: List[AttemptFeedback]

    def summary(self) -> str:
        lines = [f"Target number: {self.target}"]
        lines.append(f"Outcome: {'Victory' if self.success else 'Defeat'} in {self.attempts_used} attempts")
        for attempt, record in enumerate(self.history, start=1):
            lines.append(
                f"  Attempt {attempt}: guess {record.guess} → {record.feedback}"
                f" | Range now {record.remaining_low}–{record.remaining_high}"
            )
        return "\n".join(lines)


class NumberGuessingGame:
    def __init__(self, lower: int = 1, upper: int = 100, max_attempts: int = 5):
        if lower >= upper:
            raise ValueError("Lower bound must be smaller than upper bound")
        self.lower = lower
        self.upper = upper
        self.max_attempts = max_attempts

    @classmethod
    def from_difficulty(cls, difficulty: str) -> "NumberGuessingGame":
        presets = {
            "easy": cls(1, 50, 7),
            "standard": cls(1, 100, 6),
            "challenging": cls(1, 500, 9),
        }
        try:
            return presets[difficulty.lower()]
        except KeyError as exc:
            raise ValueError(f"Unknown difficulty '{difficulty}'. Choose from {tuple(presets)}") from exc

    def play(
        self,
        *,
        scripted_guesses: Optional[Iterable[int]] = None,
        input_func: Optional[Callable[[str], str]] = None,
        seed: Optional[int] = None,
    ) -> GuessingGameResult:
        rng = random.Random(seed)
        target = rng.randint(self.lower, self.upper)
        remaining_low, remaining_high = self.lower, self.upper
        history: List[AttemptFeedback] = []

        if scripted_guesses is not None:
            guesses_iter = iter(scripted_guesses)
            def fetch_guess(prompt: str) -> int:
                return next(guesses_iter)
        elif input_func is not None:
            def fetch_guess(prompt: str) -> int:
                return int(input_func(prompt))
        else:
            def fetch_guess(prompt: str) -> int:
                return int(input(prompt))

        for attempt in range(1, self.max_attempts + 1):
            try:
                guess = int(fetch_guess(f"Attempt {attempt}/{self.max_attempts}: "))
            except StopIteration as err:
                raise RuntimeError("Ran out of scripted guesses before the game ended") from err

            if guess == target:
                history.append(
                    AttemptFeedback(
                        guess=guess,
                        feedback="Correct!",
                        remaining_low=remaining_low,
                        remaining_high=remaining_high,
                    )
                )
                return GuessingGameResult(target, True, attempt, history)

            distance = abs(target - guess)
            threshold = max(1, (self.upper - self.lower) // 20)
            proximity = "🔥 Hot" if distance <= threshold else "❄️ Cold"
            if guess < target:
                remaining_low = max(remaining_low, guess + 1)
                hint = f"Too low ({distance} away, {proximity})"
            else:
                remaining_high = min(remaining_high, guess - 1)
                hint = f"Too high ({distance} away, {proximity})"

            history.append(AttemptFeedback(guess, hint, remaining_low, remaining_high))

        return GuessingGameResult(target, False, self.max_attempts, history)


In [ ]:
game = NumberGuessingGame.from_difficulty('standard')
result = game.play(scripted_guesses=[20, 60, 75, 88, 93, 90], seed=7)
print(result.summary())


Target number: 42
Outcome: Defeat in 6 attempts
  Attempt 1: guess 20 → Too low (22 away, ❄️ Cold) | Range now 21–100
  Attempt 2: guess 60 → Too high (18 away, ❄️ Cold) | Range now 21–59
  Attempt 3: guess 75 → Too high (33 away, ❄️ Cold) | Range now 21–59
  Attempt 4: guess 88 → Too high (46 away, ❄️ Cold) | Range now 21–59
  Attempt 5: guess 93 → Too high (51 away, ❄️ Cold) | Range now 21–59
  Attempt 6: guess 90 → Too high (48 away, ❄️ Cold) | Range now 21–59


## 3. Tic-Tac-Toe

The Tic-Tac-Toe engine keeps a detailed move history, validates plays, and exposes a simple minimax-based adviser that recommends the optimal move for the current player.

Below we can see the replay of a scripted match and query the adviser before each move.

In [ ]:
@dataclass
class MoveRecord:
    player: str
    position: Tuple[int, int]
    board_snapshot: List[List[str]]
    winner: Optional[str]
    is_draw: bool


class TicTacToeGame:
    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.current_player = "X"
        self.history: List[MoveRecord] = []

    @staticmethod
    def _check_winner(board: Sequence[Sequence[str]]) -> Optional[str]:
        lines = []
        lines.extend(board)
        lines.extend(zip(*board))
        lines.append([board[i][i] for i in range(3)])
        lines.append([board[i][2 - i] for i in range(3)])
        for line in lines:
            if line.count(line[0]) == 3 and line[0] != " ":
                return line[0]
        return None

    def available_moves(self) -> List[Tuple[int, int]]:
        return [(r, c) for r in range(3) for c in range(3) if self.board[r][c] == " "]

    def apply_move(self, position: Tuple[int, int]) -> MoveRecord:
        r, c = position
        if not (0 <= r < 3 and 0 <= c < 3):
            raise ValueError("Move out of bounds")
        if self.board[r][c] != " ":
            raise ValueError("Cell already taken")

        self.board[r][c] = self.current_player
        winner = self._check_winner(self.board)
        is_draw = winner is None and not self.available_moves()
        snapshot = [row.copy() for row in self.board]
        record = MoveRecord(self.current_player, position, snapshot, winner, is_draw)
        self.history.append(record)
        self.current_player = "O" if self.current_player == "X" else "X"
        return record

    def suggest_optimal_move(self, player: Optional[str] = None) -> Tuple[int, int]:
        player = player or self.current_player
        opponent = "O" if player == "X" else "X"

        def minimax(board: List[List[str]], active: str) -> int:
            winner = self._check_winner(board)
            if winner == player:
                return 1
            if winner == opponent:
                return -1
            if all(cell != " " for row in board for cell in row):
                return 0

            scores: List[int] = []
            for r, c in [(r, c) for r in range(3) for c in range(3) if board[r][c] == " "]:
                board[r][c] = active
                scores.append(minimax(board, "O" if active == "X" else "X"))
                board[r][c] = " "
            return (max if active == player else min)(scores)

        best_score = -math.inf
        best_move: Optional[Tuple[int, int]] = None
        temp_board = [row.copy() for row in self.board]
        for move in self.available_moves():
            r, c = move
            temp_board[r][c] = player
            score = minimax(temp_board, "O" if player == "X" else "X")
            temp_board[r][c] = " "
            if score > best_score:
                best_score = score
                best_move = move
        if best_move is None:
            raise RuntimeError("No moves available")
        return best_move

    @staticmethod
    def render(board: Sequence[Sequence[str]]) -> str:
        rows = [" | ".join(row) for row in board]
        return "\n---------\n".join(rows)


In [ ]:
game = TicTacToeGame()
scripted_moves = [(0, 0), (1, 1), (0, 1), (0, 2), (2, 0), (1, 0), (1, 2)]
for move in scripted_moves:
    suggested = game.suggest_optimal_move()
    print(f"Player {game.current_player} optimal move suggestion: {suggested}")
    record = game.apply_move(move)
    print(TicTacToeGame.render(record.board_snapshot))
    if record.winner:
        print(f"Winner: {record.winner}\n")
        break
    if record.is_draw:
        print("It's a draw!\n")
        break
else:
    final_state = game.history[-1]
    if final_state.winner:
        print(f"Winner: {final_state.winner}")
    elif final_state.is_draw:
        print("It's a draw!")


Player X optimal move suggestion: (0, 0)
X |   |  
---------
  |   |  
---------
  |   |  


Player O optimal move suggestion: (1, 1)
X |   |  
---------
  | O |  
---------
  |   |  
Player X optimal move suggestion: (0, 1)
X | X |  
---------
  | O |  
---------
  |   |  
Player O optimal move suggestion: (0, 2)
X | X | O
---------
  | O |  
---------
  |   |  
Player X optimal move suggestion: (2, 0)
X | X | O
---------
  | O |  
---------
X |   |  
Player O optimal move suggestion: (1, 0)
X | X | O
---------
O | O |  
---------
X |   |  
Player X optimal move suggestion: (1, 2)
X | X | O
---------
O | O | X
---------
X |   |  


## 4. Rock-Paper-Scissors Tournament

Two rule sets are supported: the classic trio and the extended Rock-Paper-Scissors-Lizard-Spock variant popularised by *The Big Bang Theory*.

The tournament helper aggregates round-by-round outcomes, streaks, and win rates.

In [ ]:
@dataclass
class RPSRound:
    round_number: int
    player_one_choice: str
    player_two_choice: str
    outcome: str


@dataclass
class RPSTournamentResult:
    rounds: List[RPSRound]
    score_player_one: int
    score_player_two: int
    ties: int

    def leaderboard(self) -> str:
        total = len(self.rounds)
        if total == 0:
            return "No rounds played."
        win_rate_p1 = self.score_player_one / total
        win_rate_p2 = self.score_player_two / total
        lines = [
            f"Rounds played: {total}",
            f"Player 1 wins: {self.score_player_one} ({win_rate_p1:.1%})",
            f"Player 2 wins: {self.score_player_two} ({win_rate_p2:.1%})",
            f"Ties: {self.ties}",
        ]
        return "\n".join(lines)


class RockPaperScissorsTournament:
    RULESETS = {
        "classic": {
            "rock": {"scissors"},
            "paper": {"rock"},
            "scissors": {"paper"},
        },
        "lizard-spock": {
            "rock": {"scissors", "lizard"},
            "paper": {"rock", "spock"},
            "scissors": {"paper", "lizard"},
            "lizard": {"paper", "spock"},
            "spock": {"scissors", "rock"},
        },
    }

    def __init__(self, ruleset: str = "classic") -> None:
        if ruleset not in self.RULESETS:
            raise ValueError(f"Unsupported ruleset '{ruleset}'. Choose from {tuple(self.RULESETS)}")
        self.rules = self.RULESETS[ruleset]

    def play(self, moves_player_one: Sequence[str], moves_player_two: Sequence[str]) -> RPSTournamentResult:
        if len(moves_player_one) != len(moves_player_two):
            raise ValueError("Both players must supply the same number of moves")
        rounds: List[RPSRound] = []
        score_p1 = score_p2 = ties = 0
        valid_choices = set(self.rules)

        for idx, (choice_one, choice_two) in enumerate(zip(moves_player_one, moves_player_two), start=1):
            c1 = choice_one.lower()
            c2 = choice_two.lower()
            if c1 not in valid_choices or c2 not in valid_choices:
                raise ValueError("Invalid choice supplied")
            if c1 == c2:
                outcome = "Tie"
                ties += 1
            elif c2 in self.rules[c1]:
                outcome = "Player 1"
                score_p1 += 1
            elif c1 in self.rules[c2]:
                outcome = "Player 2"
                score_p2 += 1
            else:
                outcome = "No contest"
            rounds.append(RPSRound(idx, c1, c2, outcome))

        return RPSTournamentResult(rounds, score_p1, score_p2, ties)


In [ ]:
tournament = RockPaperScissorsTournament(ruleset='lizard-spock')
p1_moves = ['rock', 'paper', 'scissors', 'lizard', 'spock']
p2_moves = ['spock', 'rock', 'lizard', 'scissors', 'paper']
result = tournament.play(p1_moves, p2_moves)
print(result.leaderboard())
for rnd in result.rounds:
    print(f"Round {rnd.round_number}: {rnd.player_one_choice} vs {rnd.player_two_choice} → {rnd.outcome}")


Rounds played: 5
Player 1 wins: 2 (40.0%)
Player 2 wins: 3 (60.0%)
Ties: 0
Round 1: rock vs spock → Player 2
Round 2: paper vs rock → Player 1
Round 3: scissors vs lizard → Player 1
Round 4: lizard vs scissors → Player 2
Round 5: spock vs paper → Player 2


## 5. Hangman

Hangman now supports a hint (reveals one unrevealed letter), ASCII art progress, and reproducible random word selection.

The scripted run demonstrates the hint mechanism followed by a sequence of guesses that completes the word.

In [ ]:
HANGMAN_PICS = [
   r"""
     +---+
     |   |
         |
         |
         |
         |
    =========
    """.strip('\n'),
    r"""
     +---+
     |   |
     O   |
         |
         |
         |
    =========
    """.strip('\n'),
    r"""
     +---+
     |   |
     O   |
     |   |
         |
         |
    =========
    """.strip('\n'),
    r"""
     +---+
     |   |
     O   |
    /|   |
         |
         |
    =========
    """.strip('\n'),
    r"""
     +---+
     |   |
     O   |
    /|\\  |
         |
         |
    =========
    """.strip('\n'),
    r"""
     +---+
     |   |
     O   |
    /|\\  |
    /    |
         |
    =========
    """.strip('\n'),
    r"""
     +---+
     |   |
     O   |
    /|\\  |
    / \  |
         |
    =========
    """.strip('\n'),
]


@dataclass
class HangmanState:
    word: str
    guessed_letters: List[str] = field(default_factory=list)
    wrong_guesses: int = 0
    revealed: List[str] = field(init=False)
    hint_used: bool = False

    def __post_init__(self) -> None:
        self.revealed = ['_' for _ in self.word]

    def display(self) -> str:
        art = HANGMAN_PICS[self.wrong_guesses]
        progress = ' '.join(self.revealed)
        guessed = ', '.join(sorted(self.guessed_letters)) or 'None'
        return f"{art}\n\nWord: {progress}\nGuessed letters: {guessed}"

    def reveal_letter(self) -> Optional[str]:
        if self.hint_used:
            return None
        for idx, char in enumerate(self.word):
            if self.revealed[idx] == '_':
                self.revealed[idx] = char
                self.hint_used = True
                return char
        return None


class HangmanGame:
    def __init__(self, words: Optional[Sequence[str]] = None, max_attempts: int = 6) -> None:
        self.words = words or [
            'analytics',
            'python',
            'data',
            'variable',
            'statistics',
            'iteration',
        ]
        self.max_attempts = max_attempts

    def play(
        self,
        *,
        scripted_guesses: Iterable[str],
        seed: Optional[int] = None,
        request_hint_at: Optional[int] = None,
    ) -> HangmanState:
        rng = random.Random(seed)
        word = rng.choice(self.words)
        state = HangmanState(word)

        for turn, guess in enumerate(scripted_guesses, start=1):
            guess = guess.lower()
            if request_hint_at and turn == request_hint_at:
                revealed = state.reveal_letter()
                if revealed:
                    print(f"Hint revealed letter: {revealed}")
            if not guess.isalpha() or len(guess) != 1:
                print(f"Ignoring invalid guess '{guess}'")
                continue
            if guess in state.guessed_letters:
                print(f"Letter '{guess}' already guessed")
                continue

            state.guessed_letters.append(guess)
            if guess in state.word:
                for idx, char in enumerate(state.word):
                    if char == guess:
                        state.revealed[idx] = char
                print(state.display())
                if '_' not in state.revealed:
                    print("Congratulations! The word was guessed correctly.")
                    return state
            else:
                state.wrong_guesses += 1
                print(state.display())
                if state.wrong_guesses >= self.max_attempts:
                    print(f"Game over. The word was '{state.word}'.")
                    return state
        return state


<>:61: SyntaxWarning: invalid escape sequence '\ '
<>:61: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_6282/884355862.py:61: SyntaxWarning: invalid escape sequence '\ '
  / \  |


In [ ]:
hangman = HangmanGame()
final_state = hangman.play(scripted_guesses=list('aeioutnrls'), seed=3, request_hint_at=3)
print('Final board:')
print(final_state.display())


     +---+
     |   |
     O   |
         |
         |
         |
    

Word: _ _ _ _ _ _
Guessed letters: a
     +---+
     |   |
     O   |
     |   |
         |
         |
    

Word: _ _ _ _ _ _
Guessed letters: a, e
Hint revealed letter: p
     +---+
     |   |
     O   |
    /|   |
         |
         |
    

Word: p _ _ _ _ _
Guessed letters: a, e, i
     +---+
     |   |
     O   |
    /|   |
         |
         |
    

Word: p _ _ _ o _
Guessed letters: a, e, i, o
     +---+
     |   |
     O   |
    /|\  |
         |
         |
    

Word: p _ _ _ o _
Guessed letters: a, e, i, o, u
     +---+
     |   |
     O   |
    /|\  |
         |
         |
    

Word: p _ t _ o _
Guessed letters: a, e, i, o, t, u
     +---+
     |   |
     O   |
    /|\  |
         |
         |
    

Word: p _ t _ o n
Guessed letters: a, e, i, n, o, t, u
     +---+
     |   |
     O   |
    /|\  |
    /    |
         |
    

Word: p _ t _ o n
Guessed letters: a, e, i, n, o, r, t, u
     +---+
     |   

## 6. Simon Says

`SimonSaysGame` gradually lengthens the colour sequence and evaluates a player's responses. A scoring breakdown summarises accuracy per round.

In [ ]:
COLORS = ['red', 'blue', 'green', 'yellow']


@dataclass
class SimonRound:
    expected_sequence: List[str]
    player_sequence: List[str]
    success: bool


@dataclass
class SimonResult:
    rounds: List[SimonRound]

    def accuracy(self) -> float:
        total = sum(len(r.expected_sequence) for r in self.rounds)
        correct = sum(
            sum(1 for expected, actual in zip(r.expected_sequence, r.player_sequence) if expected == actual)
            for r in self.rounds
        )
        return correct / total if total else 0.0

    def report(self) -> str:
        lines = [f"Overall accuracy: {self.accuracy():.1%}"]
        for idx, round_result in enumerate(self.rounds, start=1):
            status = '✅' if round_result.success else '❌'
            lines.append(
                f"Round {idx} {status} | expected {round_result.expected_sequence}"
                f" | player {round_result.player_sequence}"
            )
        return '\n'.join(lines)


class SimonSaysGame:
    def __init__(self, colors: Sequence[str] = COLORS) -> None:
        self.colors = list(colors)

    def play(
        self,
        *,
        rounds: int,
        scripted_responses: Sequence[Sequence[str]],
        seed: Optional[int] = None,
    ) -> SimonResult:
        if rounds != len(scripted_responses):
            raise ValueError("Number of scripted responses must match the number of rounds")

        rng = random.Random(seed)
        generated_sequence: List[str] = []
        results: List[SimonRound] = []

        for idx in range(rounds):
            generated_sequence.append(rng.choice(self.colors))
            player_sequence = list(scripted_responses[idx])
            success = player_sequence == generated_sequence
            results.append(SimonRound(generated_sequence.copy(), player_sequence, success))
        return SimonResult(results)


In [ ]:
simon = SimonSaysGame()
simon_result = simon.play(rounds=4, scripted_responses=[
    ['red'],
    ['red', 'blue'],
    ['red', 'blue', 'green'],
    ['red', 'blue', 'yellow', 'green'],
], seed=2)
print(simon_result.report())


Overall accuracy: 50.0%
Round 1 ✅ | expected ['red'] | player ['red']
Round 2 ❌ | expected ['red', 'red'] | player ['red', 'blue']
Round 3 ❌ | expected ['red', 'red', 'red'] | player ['red', 'blue', 'green']
Round 4 ❌ | expected ['red', 'red', 'red', 'green'] | player ['red', 'blue', 'yellow', 'green']


## 7. Number Pyramid

The pyramid helper prints a centred numeric pyramid and also returns the raw rows for downstream processing.

In [ ]:
def number_pyramid(height: int) -> List[str]:
    if height <= 0:
        raise ValueError('Height must be positive')
    width = height * 2 - 1
    rows: List[str] = []
    for level in range(1, height + 1):
        ascending = list(range(1, level + 1))
        descending = ascending[-2::-1]
        row_numbers = ascending + descending
        row = ' '.join(str(num) for num in row_numbers)
        rows.append(row.center(width * 2 - 1))
    return rows


def display_number_pyramid(height: int) -> None:
    for row in number_pyramid(height):
        print(row)


In [ ]:
display_number_pyramid(6)


          1          
        1 2 1        
      1 2 3 2 1      
    1 2 3 4 3 2 1    
  1 2 3 4 5 4 3 2 1  
1 2 3 4 5 6 5 4 3 2 1
